In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

In [3]:
spark = SparkSession.builder.appName('tratamento_camada_ouro').getOrCreate()
spark

23/10/26 22:23:30 WARN Utils: Your hostname, DESKTOP-IDQDSUJ resolves to a loopback address: 127.0.1.1; using 172.28.63.9 instead (on interface eth0)
23/10/26 22:23:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/10/26 22:23:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df_cities_skylines = spark.read.parquet('/home/rodrigo/projetos/analise_dados_youtube/data/projetos_youtube/prata/assunto_cities_skylines/extracao_data_*/estatisticas/historico_video_tratada.parquet')

In [5]:
df_cities_skylines = df_cities_skylines.select(
    'data_extracao',
      f.explode('items').alias('items')

      )

In [6]:
df_cities_skylines = df_cities_skylines.select(
    'data_extracao',
    f.col('items.id').alias('ID_VIDEO'),
    f.col('items.snippet.publishedAt').alias('DATA_PUBLICACAO'),
    f.col('items.snippet.channelId').alias('ID_CANAL'),
    f.col('items.snippet.channelTitle').alias('NM_CANAL'),
    f.col('items.snippet.categoryId').alias('ID_CATEGORIA'),
    f.col('items.snippet.title').alias('TITULO_VIDEO'),
    f.col('items.snippet.description').alias('DESCRICAO'),
    f.col('items.snippet.tags').alias('TAGS'),
    f.col('items.contentDetails.duration').alias('DURACAO_VIDEOS'),
    f.col('items.statistics.viewCount').alias('TOTAL_VISUALIZACOES'),
    f.col('items.statistics.likeCount').alias('TOTAL_LIKES'),
    f.col('items.statistics.favoriteCount').alias('TOTAL_FAVORITOS'),
    f.col('items.statistics.commentCount').alias('TOTAL_COMENTARIOS'),
)
df_cities_skylines = df_cities_skylines.withColumn('data_extracao', f.date_format(f.unix_timestamp('data_extracao', 'yyyy-MM-dd HH:mm:ss').cast('timestamp'), 'yyyy-MM-dd'))
df_cities_skylines.show(truncate=False)

+-------------+-----------+--------------------+------------------------+----------------------------+------------+---------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [7]:
df_cities_skylines.select('data_extracao').distinct().show()

+-------------+
|data_extracao|
+-------------+
|   2023-10-25|
|   2023-10-24|
|   2023-10-23|
|   2023-10-22|
|   2023-10-20|
|   2023-10-21|
|   2023-10-18|
|   2023-10-19|
|   2023-10-16|
|   2023-10-17|
|   2023-10-15|
+-------------+



In [8]:
df_cities_skylines \
    .write\
    .partitionBy('data_extracao', 'ID_CANAL')\
        .mode('overwrite').\
            parquet('/home/rodrigo/projetos/analise_dados_youtube/data/projetos_youtube/ouro/estatistica/cities_skylines.parquet')